In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course')

Hello Capstone Project Course


# <font color = purple>__1. Segmenting and Clustering Neighborhoods in Toronto__</font>

In [3]:

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# <font color = purple>__1.1 First exercice: Create the main DF__</font>

In [4]:
from pandas.io.html import read_html

link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" 
wikitable = read_html(link,attrs={'class':'wikitable sortable'}) # I'm going for the easiest way using the 'read_html' pandas method to scrap the wikitable

print('Extracted {num} wikitables'.format(num=len(wikitable))) # Check whether there are more than one table on the link


Extracted 1 wikitables


In [5]:
table = wikitable[0]
table.head() 

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
table = table[table.Borough != 'Not assigned'] # Remove rows where there is no value assigned to the column 'Borough'
table.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [7]:
table.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'},inplace=True) # Rename column as required on the casptone 
table.head(15)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [8]:
table['Neighborhood'] = table['Neighborhood'].replace('Not assigned',table['Borough']) # Replace NAN value assigned in 'Neighborhood' with the value assigned on' Borough'

table.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [9]:
table.shape # Check the shape prior to the groupBy function

(211, 3)

In [10]:
dfGroupby = table.groupby(['PostalCode','Borough'])['Neighborhood'].apply(list) # I'm creating a view of my first df using the group by function in order to group neighborhood under a unique PC
dfGroupby = dfGroupby.sample(frac=1).reset_index() # reset the index to avoid problems
dfGroupby['Neighborhood'] = dfGroupby['Neighborhood'].str.join(', ') # Add a coma to separates the n values in the column 'Nighborhood' 

dfGroupby.head()
    

,PostalCode,Borough,Neighborhood
0,M7R,Mississauga,Canada Post Gateway Processing Centre
1,M3A,North York,Parkwoods
2,M6H,West Toronto,"Dovercourt Village, Dufferin"
3,M2N,North York,Willowdale South
4,M2H,North York,Hillcrest Village


In [11]:
dfGroupby.shape # New shape of my df

(103, 3)

# <font color = purple>__1.2 Second exercice: Append the df with the lat/lon coordinates__</font>

In [12]:
url = 'http://cocl.us/Geospatial_data'
url_data = pd.read_csv(url) # loading the df from the file on my Notebook using the read_csv method
url_data.rename(columns={'Postal Code': 'PostalCode'},inplace=True) # updating the label to have similar label 

url_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
Lastdf = dfGroupby.merge(url_data) # using the merge method to join my two df on the postal code (which is made automatically using this method)
Lastdf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819
1,M3A,North York,Parkwoods,43.753259,-79.329656
2,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
3,M2N,North York,Willowdale South,43.770120,-79.408493
4,M2H,North York,Hillcrest Village,43.803762,-79.363452


# <font color = purple>__1.3 Third exercice: Cluster the neighborhoods in Toronto__</font>

In [41]:
address = 'Toronto, ON, Canada' 

geolocator = Nominatim(user_agent="toronto_explorer") # In order to define an instance of the geocoder, we need to define a user_agent
location = geolocator.geocode(address) # Extracting Toronto's coordinates using geolocator method
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [42]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Lastdf['Latitude'], Lastdf['Longitude'], Lastdf['Borough'], Lastdf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [43]:
CLIENT_ID = 'U1JPB1RU3PAUAYPENRWH3PAQB0KA4CKW2KZSXXTKDKTM5M3Y' # my Foursquare ID
CLIENT_SECRET = 'C05RBLVLNZ4V0AMBQPOSPSOV10HUJIOQI5FCUYTG3CQZORNR' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: U1JPB1RU3PAUAYPENRWH3PAQB0KA4CKW2KZSXXTKDKTM5M3Y
CLIENT_SECRET:C05RBLVLNZ4V0AMBQPOSPSOV10HUJIOQI5FCUYTG3CQZORNR


In [44]:
Lastdf.loc[0, 'Neighborhood']

'Canada Post Gateway Processing Centre'

In [45]:
neighborhood_latitude = Lastdf.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Lastdf.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Lastdf.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Canada Post Gateway Processing Centre are 43.6369656, -79.61581899999999.


In [46]:
# let's get the top 100 venues that are in York Mills West within a radius of 500 meters

Limit = 100 
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    Limit)


In [47]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dc960456f0aa2002c6ccb91'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Mississauga',
  'headerFullLocation': 'Mississauga',
  'headerLocationGranularity': 'city',
  'totalResults': 11,
  'suggestedBounds': {'ne': {'lat': 43.641465604500006,
    'lng': -79.60961278716991},
   'sw': {'lat': 43.6324655955, 'lng': -79.62202521283007}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4db8469e815445bd47d007a2',
       'name': 'Quesada Mexican Grill',
       'location': {'address': '5067 Dixie Rd',
        'crossStreet': 'at Eglinton Ave',
        'lat': 43.63691569975329,
        'lng': -79.62058021407259,
        'labeledLatLngs': [{'label': 'di

In [48]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [49]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Quesada Mexican Grill,Burrito Place,43.636916,-79.620580
1,Hilton Garden Inn,Hotel,43.638519,-79.618721
2,Courtyard Mississauga-Airport Corporate Centre...,Hotel,43.641405,-79.615543
3,Tim Hortons,Coffee Shop,43.636293,-79.615655
4,Subway,Sandwich Place,43.636801,-79.620883


In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            Limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# type your answer here

Toronto_venues = getNearbyVenues(names=Lastdf['Neighborhood'],
                                   latitudes=Lastdf['Latitude'],
                                   longitudes=Lastdf['Longitude']
                                  )


Canada Post Gateway Processing Centre
Parkwoods
Dovercourt Village, Dufferin
Willowdale South
Hillcrest Village
Forest Hill North, Forest Hill West
Cabbagetown, St. James Town
Adelaide, King, Richmond
Runnymede, Swansea
Central Bay Street
Thorncliffe Park
Don Mills North
Davisville North
Queen's Park
Woodbine Heights
Lawrence Park
Cliffcrest, Cliffside, Scarborough Village West
Clarks Corners, Sullivan, Tam O'Shanter
Bathurst Manor, Downsview North, Wilson Heights
Guildwood, Morningside, West Hill
L'Amoreaux West
First Canadian Place, Underground city
Leaside
Clairlea, Golden Mile, Oakridge
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Bayview Village
Little Portugal, Trinity
Scarborough Village
Moore Park, Summerhill East
Agincourt
Harbourfront East, Toronto Islands, Union Station
Birch Cliff, Cliffside West
East Birchmount Park, Ionview, Kennedy Park
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Maryvale, Wexford
Roselawn
Downsview, North Park, Upw

In [52]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2249, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Canada Post Gateway Processing Centre,43.636966,-79.615819,Quesada Mexican Grill,43.636916,-79.620580,Burrito Place
1,Canada Post Gateway Processing Centre,43.636966,-79.615819,Hilton Garden Inn,43.638519,-79.618721,Hotel
2,Canada Post Gateway Processing Centre,43.636966,-79.615819,Courtyard Mississauga-Airport Corporate Centre...,43.641405,-79.615543,Hotel
3,Canada Post Gateway Processing Centre,43.636966,-79.615819,Tim Hortons,43.636293,-79.615655,Coffee Shop
4,Canada Post Gateway Processing Centre,43.636966,-79.615819,Subway,43.636801,-79.620883,Sandwich Place


In [54]:
# Let's analyze each neighborhood

# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.0
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [56]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Asian Restaurant,Cosmetics Shop,Sushi Restaurant,Restaurant,Bakery,Hotel
1,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Women's Store,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Sandwich Place,Fried Chicken Joint,Japanese Restaurant,Discount Store,Fast Food Restaurant,Pizza Place,Pharmacy,Doner Restaurant
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Skating Rink,Gym,Pharmacy,Sandwich Place,Pub,Pool,Department Store,Curling Ice


In [72]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [73]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

Toronto_merged = Lastdf

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819,1.0,Hotel,Coffee Shop,Middle Eastern Restaurant,Fried Chicken Joint,Mediterranean Restaurant,Sandwich Place,Burrito Place,American Restaurant,Gym / Fitness Center,Dog Run
1,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Pool,Food & Drink Shop,Bus Stop,Donut Shop,Doner Restaurant,Drugstore,Dog Run,Curling Ice,Diner
2,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,1.0,Bakery,Pharmacy,Supermarket,Bus Stop,Bar,Park,Brewery,Gym / Fitness Center,Café,Music Venue
3,M2N,North York,Willowdale South,43.770120,-79.408493,1.0,Ramen Restaurant,Coffee Shop,Sushi Restaurant,Sandwich Place,Pizza Place,Restaurant,Café,Ice Cream Shop,Steakhouse,Hotel
4,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Fast Food Restaurant,Athletics & Sports,Mediterranean Restaurant,Pool,Dog Run,Drugstore,Donut Shop,Doner Restaurant,Dance Studio


In [86]:
Toronto_merged

finalTable = Toronto_merged.dropna()

finalTable.head(103)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819,1.0,Hotel,Coffee Shop,Middle Eastern Restaurant,Fried Chicken Joint,Mediterranean Restaurant,Sandwich Place,Burrito Place,American Restaurant,Gym / Fitness Center,Dog Run
1,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Pool,Food & Drink Shop,Bus Stop,Donut Shop,Doner Restaurant,Drugstore,Dog Run,Curling Ice,Diner
2,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,1.0,Bakery,Pharmacy,Supermarket,Bus Stop,Bar,Park,Brewery,Gym / Fitness Center,Café,Music Venue
3,M2N,North York,Willowdale South,43.770120,-79.408493,1.0,Ramen Restaurant,Coffee Shop,Sushi Restaurant,Sandwich Place,Pizza Place,Restaurant,Café,Ice Cream Shop,Steakhouse,Hotel
4,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Fast Food Restaurant,Athletics & Sports,Mediterranean Restaurant,Pool,Dog Run,Drugstore,Donut Shop,Doner Restaurant,Dance Studio
5,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,4.0,Park,Jewelry Store,Sushi Restaurant,Trail,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Curling Ice
6,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,1.0,Coffee Shop,Restaurant,Grocery Store,Italian Restaurant,Café,Pub,Pizza Place,Park,Bakery,Sandwich Place
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,1.0,Coffee Shop,Café,Steakhouse,Bar,Asian Restaurant,Cosmetics Shop,Sushi Restaurant,Restaurant,Bakery,Hotel
8,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,1.0,Coffee Shop,Café,Pizza Place,Sushi Restaurant,Italian Restaurant,Falafel Restaurant,Sandwich Place,Bar,Fish & Chips Shop,Pub
9,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1.0,Coffee Shop,Italian Restaurant,Ice Cream Shop,Sandwich Place,Burger Joint,Café,Salad Place,Japanese Restaurant,Gym / Fitness Center,Bubble Tea Shop


In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(finalTable['Latitude'], finalTable['Longitude'], finalTable['Neighborhood'], finalTable['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= rainbow[int(cluster)-1],
        fill=True,
        fill_color= rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

ValueError: cannot convert float NaN to integer